In [1]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('Qwen/Qwen2-7B-Instruct')

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

# 预训练的开源大模型
# model = AutoModelForCausalLM.from_pretrained(
#     "/mnt/data/xuhu/qwen/Qwen2-7B-Instruct",
#     torch_dtype="auto",
#     device_map="auto"
# )
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype="auto",
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_dir)

Loading checkpoint shards: 100%|██████████| 4/4 [01:24<00:00, 21.13s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import pandas as pd
dc = pd.read_csv('../dataset/en-zh.dic', sep='\t', header=None).set_index(0).to_dict()[1]

from tqdm import tqdm
lines = open('../dataset/test_en.txt').readlines()

In [4]:
result = []
for line in tqdm(lines):
    sp_words = [x for x in line.lower().split() if x in dc.keys()]
    sp_words_meaning = [dc[x] for x in sp_words]

    sp_prompt = '文章字符为：'
    if len(sp_words) > 0:
        for x, y in zip(sp_words, sp_words_meaning): 
            sp_prompt += f'{x} 翻译为 {y}; '
    # 主要任务
    messages = [
        {"role": "system", "content": "将英文翻译为中文，不要有其他输出，直接输出翻译后的文本。保留特殊单词的翻译。"},
    ]

    # 人工的词典的规则
    if len(sp_prompt) > 0:
        messages.append({"role": "user", "content": sp_prompt})
    messages.append({"role": "user", "content": f"待翻译文本（从英文翻译为中文）：{line}"})
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    result_line = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    result.append(result_line)
    
with open('../results/qwen_submit.txt', 'w') as f:
    for line in result:
        line = line.strip().replace('\n', '')
        print(line)
        f.write(line + '\n')

100%|██████████| 1000/1000 [16:46<00:00,  1.01s/it]

罐头厂消失了。污染得到了缓解。
Handspring Puppet Co.: 天才木偶艺术《战马》
但是现在我们想让你让乔伊做一些动作。
这就是我们在青少年和孩子在课堂上、桌子下以及向朋友发短信时所看到的情况。
当宣布他们要为乌拉圭境内的每一个孩子提供10万台笔记本时，OLPC顿时名声大噪。
在右边这里，我们称之为史蒂文·斯皮尔伯格纪念模块。
是的，有关资金应该如何分配的问题存在，而且这个问题还在被重新考虑之中。
一台是印刷机，看起来像是鲁贝·戈德堡设计的机器。
有一个项目是由国家科学基金会资助的——送了一批扫描仪，美国的图书馆本该把书籍寄过去。
因此，我认为计算机互联网的发明是基于先前证明的、生物上成功的模式的一种必然结果。
美国国防部的小儿麻痹症专家厄尔·肯恩博士指出，任何选择性指数大于二的化合物都是有效的。
当你从一个用阿司匹林治疗关节炎的世界过渡到一个在病情严重时可以进行髋关节置换或膝关节置换，让你数年甚至数十年免于残疾的世界时，这种巨大的变化，你是否会觉得花费4万美元进行髋关节置换，取代原本只需10美分的阿司匹林，会显得更为昂贵呢？
它会通过邮件送达；它可能来自澳大利亚、夏威夷或者佛罗里达，你可以引进那只蜂王。
实际上，如果你们住在西海岸，可以去这些网站，他们真的在寻找人来寻找一些胡蜂，因为我们认为有些已经灭绝了。或者有些，数量减少了。
许多昆虫被植物用作传粉运输的媒介，但不仅仅局限于昆虫；鸟类、爬行动物和蝙蝠、老鼠等哺乳动物通常也被用来运输花粉。
我骑车时吹口哨。我在到处吹口哨。
我想要提醒大家的是，三年前在TED上保罗·麦克雷迪曾说过一个极棒的观点。
这就是我们的说法，对吧，伙计们？
如果有人遮挡了银河系，它会看起来是这样的。
纳林·纳达基挑战了我们对树木和监狱的看法——她说，两者都比我们想象的更动态。通过与华盛顿州的合作，她将科学课程和保护项目带到囚犯中，产生了意想不到的结果。
然后这种偶尔的同情，我们不是在说——它永远不会保持偶尔性。
但如果不具备同情心，又该如何表现出同情心呢？
Oliver Sacks: 关于我们心智的幻觉揭示了什么
查尔斯·卢林，祖父，看到了成百上千种不同的形象，各种各样的风景。
查尔斯·邦纳德在250年前这么说——他想知道，这些幻觉是如何产生的，正如他所说，大脑这部机器是如何创造出这个心灵的舞台的。
看起来就像是小串珠子基本